In [7]:
from datascience import *
import numpy as np
import pandas as pd
%matplotlib inline

### Question 1

In [107]:
#This part accounts for all TAZ travel time that arrives at ATT Stadium
travel_time_data = Table.read_table('../minilab4/sf_oak_traveltimes_bymode.csv')
Metallica = travel_time_data.where("destination", 110)\
    .select("origin", "destination", "shared ride (2 people)", "walk-transit-walk")
Metallica

origin,destination,shared ride (2 people),walk-transit-walk
1,110,8.56,24.5
2,110,8.51,27.5
3,110,8.96,21.88
4,110,7.94,25.8
5,110,7.6,19.27
6,110,8.51,24.73
7,110,8.02,21.99
8,110,8.36,21.56
9,110,8.94,31.8
10,110,8.67,23.83


In [98]:
#The population table
taz_data = Table.read_table('tazData.csv').where('TOTPOP', are.above(0)).select("ZONE", "AGE2044", "AGE4564")
taz_data

ZONE,AGE2044,AGE4564
1,27,29
2,88,93
3,159,169
4,46,48
5,308,327
6,2183,1039
7,2474,2052
8,2660,3562
9,4180,2677
10,1216,1152


In [100]:
#Creates teh table that gives both population and travel time information
pop_traffic = taz_data.join("ZONE", Metallica, "origin")
pop_traffic

ZONE,AGE2044,AGE4564,destination,shared ride (2 people),walk-transit-walk
1,27,29,110,8.56,24.5
2,88,93,110,8.51,27.5
3,159,169,110,8.96,21.88
4,46,48,110,7.94,25.8
5,308,327,110,7.6,19.27
6,2183,1039,110,8.51,24.73
7,2474,2052,110,8.02,21.99
8,2660,3562,110,8.36,21.56
9,4180,2677,110,8.94,31.8
10,1216,1152,110,8.67,23.83


In [102]:
#Round the answer because people are indivisible!
attendees = round(sum(pop_traffic.column("AGE2044")*0.02 + pop_traffic.column("AGE4564")*0.005))
attendees

13957.0

### Question 2

In [113]:
#Adjusts travel time for driving based on the homework's premises 
Metallica_adjusted = pop_traffic.with_columns("shared ride (2 people)", pop_traffic.column("shared ride (2 people)")+25)
Metallica_adjusted

ZONE,AGE2044,AGE4564,destination,shared ride (2 people),walk-transit-walk
1,27,29,110,33.56,24.5
2,88,93,110,33.51,27.5
3,159,169,110,33.96,21.88
4,46,48,110,32.94,25.8
5,308,327,110,32.6,19.27
6,2183,1039,110,33.51,24.73
7,2474,2052,110,33.02,21.99
8,2660,3562,110,33.36,21.56
9,4180,2677,110,33.94,31.8
10,1216,1152,110,33.67,23.83


In [114]:
#Creates an array that contains the shortest time to travel to ATT stadium
#Other methods that filter out the "prefer transit" TAZs are also valid
fast = []
for i in np.arange(Metallica_adjusted.num_rows):
    transit_time = abs(Metallica_adjusted.column("walk-transit-walk").item(i))
    drive_time = abs(Metallica_adjusted.column("shared ride (2 people)").item(i))
    fast.append(min(transit_time, drive_time))


In [115]:
#Append the shortest travel times to the table
Metallica_timed = Metallica_adjusted.with_columns("shortest_time", fast)

#Turns the array into an True/False column that indicates whether driving is preferred
Metallica_drive = Metallica_timed.with_column("drive?", Metallica_timed.column("shared ride (2 people)") == Metallica_timed.column("shortest_time"))
Metallica_drive

ZONE,AGE2044,AGE4564,destination,shared ride (2 people),walk-transit-walk,shortest_time,drive?
1,27,29,110,33.56,24.5,24.5,False
2,88,93,110,33.51,27.5,27.5,False
3,159,169,110,33.96,21.88,21.88,False
4,46,48,110,32.94,25.8,25.8,False
5,308,327,110,32.6,19.27,19.27,False
6,2183,1039,110,33.51,24.73,24.73,False
7,2474,2052,110,33.02,21.99,21.99,False
8,2660,3562,110,33.36,21.56,21.56,False
9,4180,2677,110,33.94,31.8,31.8,False
10,1216,1152,110,33.67,23.83,23.83,False


In [116]:
#Get the travel distances for driving--we don't count distances travlled by transit
travel_dist_data = Table.read_table('sf_oak_traveldists_bymode.csv').where("destination", 110)\
    .select("origin", "destination", "shared ride (2 people)")
travel_dist_data

origin,destination,shared ride (2 people)
1,110,2.01
2,110,2.06
3,110,2.23
4,110,1.81
5,110,1.65
6,110,2.03
7,110,1.85
8,110,1.99
9,110,2.2
10,110,2.12


In [117]:
#Joins the table with travel distance data
Metal_dist = Metallica_drive.join("ZONE", travel_dist_data, "origin")
Metal_dist

ZONE,AGE2044,AGE4564,destination,shared ride (2 people),walk-transit-walk,shortest_time,drive?,destination_2,shared ride (2 people)_2
1,27,29,110,33.56,24.5,24.5,False,110,2.01
2,88,93,110,33.51,27.5,27.5,False,110,2.06
3,159,169,110,33.96,21.88,21.88,False,110,2.23
4,46,48,110,32.94,25.8,25.8,False,110,1.81
5,308,327,110,32.6,19.27,19.27,False,110,1.65
6,2183,1039,110,33.51,24.73,24.73,False,110,2.03
7,2474,2052,110,33.02,21.99,21.99,False,110,1.85
8,2660,3562,110,33.36,21.56,21.56,False,110,1.99
9,4180,2677,110,33.94,31.8,31.8,False,110,2.2
10,1216,1152,110,33.67,23.83,23.83,False,110,2.12


In [118]:
#Filters so that only the TAZs that prefer driving remains
drive = Metal_dist.where("drive?", True)
drive

ZONE,AGE2044,AGE4564,destination,shared ride (2 people),walk-transit-walk,shortest_time,drive?,destination_2,shared ride (2 people)_2
42,4774,827,110,40.04,48.91,40.04,True,110,5.39
43,2754,1262,110,39.14,43.62,39.14,True,110,4.42
49,1531,1451,110,37.96,42.8,37.96,True,110,3.86
50,1841,1161,110,39.29,40,39.29,True,110,4.42
51,1787,858,110,40.37,40.93,40.37,True,110,4.89
52,2117,415,110,42.67,60.25,42.67,True,110,5.69
53,117,50,110,47.64,53.77,47.64,True,110,5.91
54,2147,2022,110,45.91,55.17,45.91,True,110,7.67
55,1147,1166,110,44.32,45.14,44.32,True,110,7.03
56,24,40,110,44.61,51.27,44.61,True,110,7.07


In [119]:
#VMT_total = number of people who drives to concert * the distance they travel in the vehicle
VMT_total = sum((0.02*drive.column("AGE2044") + 0.005*drive.column("AGE4564"))*drive.column("shared ride (2 people)_2"))
VMT_total

51719.46379999996

In [120]:
#Divide 2 since everyone is counted twice in a carpool!
VMT = round(VMT_total/2)
VMT

25860.0